In [32]:
from llama_index.core import VectorStoreIndex, Settings, Document
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors.llm_selectors import LLMSingleSelector
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from typing import Optional, List, Mapping, Any
from llama_index.core.vector_stores import FilterCondition, MetadataFilter, MetadataFilters
import logging
from dotenv import load_dotenv
import os

In [24]:
# Load environment variables
load_dotenv()
# Quality Control Data
qc_data = [
    "Defect Class A: Critical - Immediate production halt required",
    "Defect Class B: Major - Requires rework within 24 hours",
    "Defect Class C: Minor - Log and address in next maintenance cycle",
    "Common Defect: Misaligned bearings in Machine X",
    "Root Cause Analysis: Overheating due to insufficient lubrication"
]

# Maintenance Data
maint_data = [
    "Machine Type X: Lubricate bearings every 200 operating hours",
    "Machine Type Y: Calibrate sensors weekly",
    "Machine Type Z: Replace filters monthly",
    "Maintenance Log: Machine X last serviced on 2024-01-15",
    "Repair History: Machine Y had motor replaced on 2023-12-01"
]

# Convert to documents with metadata
qc_docs = [Document(text=text, metadata={"domain": "quality"}) for text in qc_data]
maint_docs = [Document(text=text, metadata={"domain": "maintenance"}) for text in maint_data]

In [25]:

# Initialize Groq LLM
groq_api_key = os.getenv("GROQ_API_KEY")
llm = Groq(model="mixtral-8x7b-32768", api_key=groq_api_key)

# Configure global settings
Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Initialize reranker
reranker = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-6-v2",
    top_n=2
)

e:\LLama-Index\venv\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\huggingface_cache\models--cross-encoder--ms-marco-MiniLM-L-6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [26]:
# Build indices with reranking
qc_index = VectorStoreIndex.from_documents(qc_docs)
qc_engine = qc_index.as_query_engine(
    node_postprocessors = [reranker],
    similarity_top_k = 3
)

maint_index = VectorStoreIndex.from_documents(maint_docs)
maint_engine = maint_index.as_query_engine(
    node_postprocessors = [reranker],
    similarity_top_k = 3
)

In [27]:
# Create query engine tools
qc_tool = QueryEngineTool(
    query_engine= qc_engine,
    metadata= ToolMetadata(
        name="quality_control",
        description="Quality control protocols and defect analysis"
    )
)
maint_tool = QueryEngineTool(
    query_engine=maint_engine,
    metadata=ToolMetadata(
        name="maintenance",
        description="Machine maintenance schedules and repair histories"
    )
)

In [28]:
# Create ReAct agent
Agent=ReActAgent.from_tools(
    tools =[qc_tool,maint_tool],
    verbose= True
)
# Configure router
router = RouterQueryEngine(
    selector= LLMSingleSelector.from_defaults(),
    query_engine_tools=[qc_tool,maint_tool]
)
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("manufacturing_agent")

In [29]:
# Test agent
response = Agent.chat("Machine X has misaligned bearings. What should I do?")
logger.info(f"Agent Response: {response}")

> Running step 54e71d04-b32d-4e6d-967e-3ea28d36430e. Step input: Machine X has misaligned bearings. What should I do?


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: maintenance
Action Input: {'input': 'Machine X'}


Batches: 100%|██████████| 1/1 [00:00<00:00,  8.76it/s]
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Observation: Based on the information provided, Machine X is a type of machine that requires its bearings to be lubricated every 200 operating hours. The last service date for Machine X was on 2024-01-15, according to the maintenance log.
> Running step 87dcb6ed-f457-4772-95f7-1ef113258f54. Step input: None


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:manufacturing_agent:Agent Response: Based on the maintenance records, Machine X was last serviced on 2024-01-15. If the misaligned bearings are due to lack of maintenance, then it has been approximately 200 operating hours since the last service. Therefore, it is recommended to perform maintenance on Machine X to realign the bearings and lubricate them.


Thought: I can use the information from the maintenance tool to determine if the misaligned bearings could be due to lack of maintenance.
Answer: Based on the maintenance records, Machine X was last serviced on 2024-01-15. If the misaligned bearings are due to lack of maintenance, then it has been approximately 200 operating hours since the last service. Therefore, it is recommended to perform maintenance on Machine X to realign the bearings and lubricate them.


In [30]:
# Add severity metadata
for doc in qc_docs:
    doc.metadata["severity"] = "high" if "Class A" in doc.text else "medium"

In [33]:
# Rebuild the index with updated metadata
qc_index = VectorStoreIndex.from_documents(qc_docs)

Batches: 100%|██████████| 1/1 [00:00<00:00,  6.23it/s]


In [35]:
# Define metadata filter
metadata_filter = MetadataFilters(
    filters=[
        MetadataFilter(key="severity", value="high", condition="eq")  # Use "eq" instead of FilterCondition.EQ
    ]
)


In [36]:
# Configure query engine with metadata filtering
qc_engine = qc_index.as_query_engine(
    similarity_top_k=5,
    filters=metadata_filter,  # Apply metadata filter here
    node_postprocessors=[reranker]
)

In [37]:
# Test metadata filtering
results = qc_engine.query("Show critical defects")
logger.info(f"Filtered Results: {results}")

Batches: 100%|██████████| 1/1 [00:00<00:00, 57.84it/s]
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:manufacturing_agent:Filtered Results: Defects classified as "Class A" are considered critical. These defects require immediate attention and may necessitate a production halt until they are resolved.
